In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
import seaborn as sns
import statistics
from tqdm import tqdm

In [3]:
def log_beta(beta0, i, c=1):
    beta = np.log(np.exp(c*beta0)+i)/c
    return beta

In [5]:
bounds = Bounds([0], [np.inf])
def langevin(x0, d, func, grad, maxiter = 100, eta0 = 0.001, beta0 = 1, beta_schedule = log_beta, c=1):
    """
    This code implements Gradient Langevin Algorithm with exact linesearch on the step size eta

    Args:
        x0 (numpy darray): initial point
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        maxiter (int): maximum number of iterations
        eta0 (float): initial value for eta
        beta0 (float): initial value for beta
        beta_schedule (Callable): annealing schedule for temperature, starting with beta0 and ending with beta1
        c (float): constant in logarithmic annealing schedule

    Output:
        f_list (numpy darray): the list of function values for each iteration
        x_list (numpy darray): the list of x values for each iteration
    """
    x_list = np.zeros((maxiter,d))
    x_list[0,:] = x0
    f_list = np.zeros((maxiter,))
    f_list[0] = func(x0)
    for i in range(1,maxiter):
        epsilon = np.random.normal(0, 1, d)
        beta = beta_schedule(beta0, i, c=c)
        def objective_function(eta):
            return func(x_list[i-1,:]- eta*grad(x_list[i-1,:]) + np.sqrt(2*eta/beta)*epsilon)
        # perform exact linesearch
        result = minimize(objective_function, eta0, method = "SLSQP", bounds=bounds)
        eta = result.x
        x_list[i,:] = x_list[i-1,:] - eta*grad(x_list[i-1,:]) + np.sqrt(2*eta/beta)*epsilon
        f_list[i] = func(x_list[i,:])
    return f_list, x_list

In [7]:
def gradient_descent(x0, d, func, grad, maxiter = 100, eta0 = 0.001):
    """
    This code implements Gradient Descent with exact linesearch on the step size eta

    Args:
        x0 (numpy darray): initial point
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        maxiter (int): maximum number of iterations
        eta0 (float): initial value for eta

    Output:
        f_list (numpy darray): the list of function values for each iteration
        x_list (numpy darray): the list of x values for each iteration
    """
    x_list = np.zeros((maxiter,d))
    x_list[0,:] = x0
    f_list = np.zeros((maxiter,))
    f_list[0] = func(x0)
    for i in range(1,maxiter):
        def objective_function(eta):
            return func(x_list[i-1,:]- eta*grad(x_list[i-1,:]))
        # perform exact linesearch
        result = minimize(objective_function, eta0, method = "SLSQP", bounds=bounds)
        eta = result.x
        x_list[i,:] = x_list[i-1,:] - eta*grad(x_list[i-1,:])
        f_list[i] = func(x_list[i,:])
    return f_list, x_list

In [13]:
def langevin_agd_x_noise(x0, d, func, grad, theta, maxiter = 100, L = None, C1 = 1):
    """
    This code implements Accelerated Langevin with Perturbed x

    Args:
        x0 (numpy darray): initial point
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        maxiter (int): maximum number of iterations
        theta (float): momentum parameter
        L (float): Lipschitz constant
        C1 (float): constant for scaling beta

    Output:
        f_list (numpy darray): the list of function values for each iteration
        x_list (numpy darray): the list of x values for each iteration
    """
    x_list = np.zeros((maxiter,d))
    y_list = np.zeros((maxiter,d))
    f_list = np.zeros((maxiter,))
    x_list[0,:] = x0
    f_list[0] = func(x_list[0,:])
    for i in range(maxiter-1):
        eps = np.random.normal(0, 1, size = (d,))
        if L is not None:
            eta = min(C0/np.sqrt(i+1), theta/(6*L))
        else:
            eta = C0/((i+1)**(2/3))
        beta = C1*(i+1)**(2/3)
        xi = x_list[i,:] + np.sqrt(2*eta/beta)*eps
        y_list[i+1,:] = xi - eta*grad(xi)
        x_list[i+1,:] = y_list[i+1,:] + (1-theta)*(y_list[i+1,:]-y_list[i,:])
        f_list[i+1] = func(x_list[i+1,:])
    return f_list, x_list

In [15]:
def langevin_agd_y_noise(x0, d, func, grad, theta, maxiter = 100, L = None, C1 = 1):
    """
    This code implements Accelerated Langevin with Perturbed y

    Args:
        x0 (numpy darray): initial point
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        maxiter (int): maximum number of iterations
        theta (float): momentum parameter
        L (float): Lipschitz constant
        C1 (float): constant for scaling beta

    Output:
        f_list (numpy darray): the list of function values for each iteration
        x_list (numpy darray): the list of x values for each iteration
    """
    x_list = np.zeros((maxiter,d))
    y_list = np.zeros((maxiter,d))
    f_list = np.zeros((maxiter,))
    x_list[0,:] = x0
    f_list[0] = func(x_list[0,:])
    for i in range(maxiter-1):
        eps = np.random.normal(0, 1, size = (d,))
        if L is not None:
            eta = min(C0/np.sqrt(i+1), theta/(3*L))
        else:
            eta = C0/((i+1)**(2/3))
        beta = C1*(i+1)**(2/3)
        y_list[i+1,:] = x_list[i,:] - eta*grad(x_list[i,:]) + np.sqrt(2*eta/beta)*eps
        x_list[i+1,:] = y_list[i+1,:] + (1-theta)*(y_list[i+1,:]-y_list[i,:])
        f_list[i+1] = func(x_list[i+1,:])
    return f_list, x_list

In [19]:
def plot(d, func, grad, maxiter=400, eta0_gld=1e-2, eta0_gd=1e-2, beta0=1, beta_schedule=log_beta, c=1, var=1, C0=1, C1=1, theta=0.9):
    """
    This code compares Gradient Langevin, Gradient Descent and Accelerated Langevin by:
    1) plotting the average convergence curve in 100 runs;
    2) computing Q1, median and Q3 of the final value of both algorithms.

    Args:
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        maxiter (int): maximum number of iterations
        eta0_gld (float): initial value for eta used in Gradient Langevin
        eta0_gd (float): initial value for eta used in Gradient Descent
        beta0 (float): initial value for beta
        beta_schedule (Callable): annealing schedule for temperature, starting with beta0
        c (float): parameter in annealing schedule
        var (float): variance of the distribution where the initial point is sampled from normal distribution
        C0 (float): constant for scaling eta in Accelerated Langevin
        C1 (float): constant for scaling beta in Accelerated Langevin
        theta (float): momentum parameter
    
    """
    gld_result = None
    gd_result = None
    perturb_x_result = None
    perturb_y_result = None
    x = np.zeros((100,d))
    gld_x = np.zeros((100,d))
    gd_x = np.zeros((100,d))
    perturb_x = np.zeros((100,d))
    perturb_y = np.zeros((100,d))
    n_x = np.zeros((100,d))
    for i in tqdm(range(100)):
        x0 = np.random.normal(0,var, size=(d,))
        x[i,:] = x0
        f_list_gld, x_list_gld = langevin(x0, d, func, grad, maxiter=maxiter, eta0 = eta0_gld, beta0 = beta0, beta1=beta1, beta_schedule = beta_schedule, c=c)
        f_list_grad, x_list_gd = gradient_descent(x0, d, func, grad, maxiter=maxiter, eta0 = eta0_gd)
        f_list_perturb_x, x_list_perturb_x = langevin_agd_x_noise(x0, d, func, grad, theta = theta, maxiter=maxiter, C0 = C0, C1 = C1)
        f_list_perturb_y, x_list_perturb_y = langevin_agd_y_noise(x0, d, func, grad, theta = theta, maxiter=maxiter, C0 = C0, C1 = C1)
        
        gld_x[i,:] = x_list_gld[-1,:]
        gd_x[i,:] = x_list_gd[-1,:]
        perturb_x[i,:] = x_list_perturb_x[-1,:]
        perturb_y[i,:] = x_list_perturb_y[-1,:]
        
        f_list_gld = np.array(f_list_gld)
        f_list_grad = np.array(f_list_grad)
        f_list_perturb_x = np.array(f_list_perturb_x)
        f_list_perturb_y = np.array(f_list_perturb_y)
        f_list_gld = f_list_gld.reshape((maxiter,1))
        f_list_grad = f_list_grad.reshape((maxiter,1))
        f_list_perturb_x = f_list_perturb_x.reshape((maxiter,1))
        f_list_perturb_y = f_list_perturb_y.reshape((maxiter,1))
        
        if i == 0:
            gld_result = f_list_gld
            gd_result = f_list_grad
            perturb_x_result = f_list_perturb_x
            perturb_y_result = f_list_perturb_y
            
        else:
            gld_result = np.concatenate((gld_result, f_list_gld), axis = 1)
            gd_result = np.concatenate((gd_result, f_list_grad), axis = 1)
            perturb_x_result = np.concatenate((perturb_x_result, f_list_perturb_x), axis = 1)
            perturb_y_result = np.concatenate((perturb_y_result, f_list_perturb_y), axis = 1)

    # Extract final output
    gld_final = gld_result[-1,:]
    gd_final = gd_result[-1,:]
    perturb_x_final = perturb_x_result[-1,:]
    perturb_y_final = perturb_y_result[-1,:]

    mean_gld = np.mean(gld_result, axis=1)
    std_dev_gld = np.std(gld_result, axis=1)
    mean_gd = np.mean(gd_result, axis=1)
    std_dev_gd = np.std(gd_result, axis=1)
    mean_perturb_x = np.mean(perturb_x_result, axis=1)
    std_dev_perturb_x = np.std(perturb_x_result, axis=1)
    mean_perturb_y = np.mean(perturb_y_result, axis=1)
    std_dev_perturb_y = np.std(perturb_y_result, axis=1)

    gld_mean = statistics.mean(gld_final)
    q1, q2, q3 = statistics.quantiles(gld_final, n=4)  # q2 == median
    print("Q1, Q2(median), Q3, mean for Gradient Langevin:", q1, q2, q3, gld_mean)
    gd_mean = statistics.mean(gd_final)
    q1, q2, q3 = statistics.quantiles(gd_final, n=4)  # q2 == median
    print("Q1, Q2(median), Q3, mean for Gradient Descent:", q1, q2, q3, gd_mean)
    perturb_x_mean = statistics.mean(perturb_x_final)
    q1, q2, q3 = statistics.quantiles(perturb_x_final, n=4)  # q2 == median
    print("Q1, Q2(median), Q3, mean for Perturb x:", q1, q2, q3, perturb_x_mean)
    perturb_y_mean = statistics.mean(perturb_y_final)
    q1, q2, q3 = statistics.quantiles(perturb_y_final, n=4)  # q2 == median
    print("Q1, Q2(median), Q3, mean for Perturb y:", q1, q2, q3, perturb_y_mean)
    n_mean = statistics.mean(n_final)

    if d == 2:
        x_s = np.linspace(-4*var, 4*var, 400)
        y_s = np.linspace(-4*var, 4*var, 400)
        X, Y = np.meshgrid(x_s, y_s)
        XY = np.stack([X.ravel(), Y.ravel()], axis=1) 
        Z = np.array([func(xy) for xy in XY])
        Z = Z.reshape(X.shape)
        
        
        plt.plot(np.arange(0,maxiter), np.log(mean_gld), label = "Gradient Langevin")
        plt.plot(np.arange(0,maxiter), np.log(mean_gd), label = "Gradient Descent")
        plt.plot(np.arange(0,maxiter), np.log(mean_perturb_x), label = "Perturb x")
        plt.plot(np.arange(0,maxiter), np.log(mean_perturb_y), label = "Perturb y")
        # plt.plot(np.arange(0,maxiter), np.log(mean_n+5.162), label = "Nesterov")
        # plt.ylim(0,5)
        plt.xlabel('Number of Gradient Evaluations')
        plt.ylabel('log(f(x_k)-f*)')
        plt.legend()
        plt.savefig('benchmark.jpg')
        
    else:
        plt.plot(np.arange(0,maxiter), np.log10(mean_gld), label = "Gradient Langevin")
        plt.plot(np.arange(0,maxiter), np.log10(mean_gd), label = "Gradient Descent")
        plt.plot(np.arange(0,maxiter), np.log10(mean_perturb_x), label = "Perturb x")
        plt.plot(np.arange(0,maxiter), np.log10(mean_perturb_y), label = "Perturb y")
        plt.legend()
        plt.xlim(-4*var, 4*var)
        plt.ylim(-4*var, 4*var)

In [21]:
def multistart(d, 
            func, 
            grad, 
            maxiter=400, 
            eta0_gld=1e-2, 
            eta0_gd=1e-2, 
            beta0=1, 
            beta_schedule=log_beta, 
            c=1, 
            var=1, 
            C0=1, 
            C1=1, 
            theta=0.9,
            x_min=0,
            x_max=3,
            y_min=0,
            y_max=3):
    """
    This code compares Gradient Langevin, Gradient Descent and Accelerated Langevin by combining them with multistart method

    Args:
        d (int): dimension of the objective function
        func (Callable): objective function
        grad (Callable): gradient of objective function
        maxiter (int): maximum number of iterations
        eta0_gld (float): initial value for eta used in Gradient Langevin
        eta0_gd (float): initial value for eta used in Gradient Descent
        beta0 (float): initial value for beta
        beta_schedule (Callable): annealing schedule for temperature, starting with beta0
        c (float): parameter in annealing schedule
        var (float): variance of the distribution where the initial point is sampled from normal distribution
        C0 (float): constant for scaling eta in Accelerated Langevin
        C1 (float): constant for scaling beta in Accelerated Langevin
        theta (float): momentum parameter
        x_min (float): min of x
        x_max (float): max of x
        y_min (float): min of y
        y_max (float): max of y
    
    """
    gld_result = None
    gd_result = None
    perturb_x_result = None
    perturb_y_result = None
    x = np.zeros((100,d))
    gld_x = np.zeros((100,d))
    gd_x = np.zeros((100,d))
    perturb_x = np.zeros((100,d))
    perturb_y = np.zeros((100,d))
    n_x = np.zeros((100,d))
    # Number of points you want
    n_points = 100  
    
    
    # Uniform sampling in each dimension
    x_coords = np.random.uniform(x_min, x_max, n_points)
    y_coords = np.random.uniform(y_min, y_max, n_points)
    # Combine into (n_points, 2) array
    points = np.column_stack((x_coords, y_coords))

    for i in tqdm(range(100)):
        x0 = points[i,:]
        x[i,:] = x0
        f_list_gld, x_list_gld = langevin(x0, d, func, grad, maxiter=maxiter, eta0 = eta0_gld, beta0 = beta0, beta1=beta1, beta_schedule = beta_schedule, c=c)
        f_list_grad, x_list_gd = gradient_descent(x0, d, func, grad, maxiter=maxiter, eta0 = eta0_gd)
        f_list_perturb_x, x_list_perturb_x = langevin_agd_x_noise(x0, d, func, grad, theta = theta, maxiter=maxiter, C0 = C0, C1 = C1)
        f_list_perturb_y, x_list_perturb_y = langevin_agd_y_noise(x0, d, func, grad, theta = theta, maxiter=maxiter, C0 = C0, C1 = C1)
        
        gld_x[i,:] = x_list_gld[-1,:]
        gd_x[i,:] = x_list_gd[-1,:]
        perturb_x[i,:] = x_list_perturb_x[-1,:]
        perturb_y[i,:] = x_list_perturb_y[-1,:]
        f_list_gld = np.array(f_list_gld)
        f_list_grad = np.array(f_list_grad)
        f_list_perturb_x = np.array(f_list_perturb_x)
        f_list_perturb_y = np.array(f_list_perturb_y)
        
        f_list_gld = f_list_gld.reshape((maxiter,1))
        f_list_grad = f_list_grad.reshape((maxiter,1))
        f_list_perturb_x = f_list_perturb_x.reshape((maxiter,1))
        f_list_perturb_y = f_list_perturb_y.reshape((maxiter,1))
        
        if i == 0:
            gld_result = f_list_gld
            gd_result = f_list_grad
            perturb_x_result = f_list_perturb_x
            perturb_y_result = f_list_perturb_y

        else:
            gld_result = np.concatenate((gld_result, f_list_gld), axis = 1)
            gd_result = np.concatenate((gd_result, f_list_grad), axis = 1)
            perturb_x_result = np.concatenate((perturb_x_result, f_list_perturb_x), axis = 1)
            perturb_y_result = np.concatenate((perturb_y_result, f_list_perturb_y), axis = 1)

    # Extract final output
    gld_final = gld_result[-1,:]
    gd_final = gd_result[-1,:]
    perturb_x_final = perturb_x_result[-1,:]
    perturb_y_final = perturb_y_result[-1,:]

    min_gld = np.min(gld_final)
    min_gd = np.min(gd_final)
    min_perturb_x = np.min(perturb_x_final)
    min_perturb_y = np.min(perturb_y_final)
    

    print("Min for GLD", min_gld)
    print("Min for GD", min_gd)
    print("Min for perturb x", min_perturb_x)
    print("Min for perturb y", min_perturb_y)